In [1]:
import pandas as pd 
from pandasql import sqldf
data = pd.read_csv('IndieZ.csv')

In [2]:
sqldf('''
SELECT 
    nop.version,
    nop.user as num_of_player,
    notc.user as tut_completed,
    (CAST(notc.user as FLOAT) / CAST(nop.user as FLOAT)) *100 
        as tut_complete_percentage
FROM (
    SELECT 
        version,
        COUNT(DISTINCT user) as user 
    FROM 
        data
    GROUP BY 
        version
) as nop
JOIN (
    SELECT 
        version,
        COUNT(DISTINCT user) as user
    FROM 
        data
    WHERE 
        quantity = -2 
    GROUP BY 
        version
) as notc
ON nop.version = notc.version 
''')

,version,num_of_player,tut_completed,tut_complete_percentage
0,1.5.2,6671,6341,95.053215
1,1.6.0,6929,6571,94.833309


In [3]:
sqldf('''
SELECT 
    tut_complete.version,
    tut_complete.user as tut_completed_user,
    lv_1_lost.user as lv_1_lost_player,
    ROUND(CAST(lv_1_lost.user as FLOAT) / CAST(tut_complete.user as FLOAT) * 100, 2) as percentage
FROM (
    SELECT 
        version,
        COUNT(DISTINCT user) as user 
    FROM 
        data
    WHERE 
        quantity = -2 
    GROUP BY 
        version
) as tut_complete
JOIN (
    SELECT 
        version,
        COUNT(DISTINCT user) as user
    FROM 
        data
    WHERE 
        user IN (
            SELECT  
                user
            FROM 
                data
            WHERE 
                event_name = 'tutorial' AND quantity = -2) 
        AND level = 1 AND win = 0
    GROUP BY 
        version
) as lv_1_lost
ON tut_complete.version = lv_1_lost.version 
''')

,version,tut_completed_user,lv_1_lost_player,percentage
0,1.5.2,6341,1561,24.62
1,1.6.0,6571,1373,20.89


In [4]:
sqldf('''
SELECT 
    tut_complete.version,
    tut_complete.player as num_of_lv_2_player,
    lost.player as lv_2_lost_player,
    ROUND(CAST(lost.player as FLOAT) / CAST(tut_complete.player as FLOAT) * 100, 2) as percentage
FROM (
    SELECT 
        version,
        COUNT(DISTINCT user) as player 
    FROM 
        data
    WHERE 
        level = 2
    GROUP BY 
        version
) as tut_complete
JOIN (
    SELECT 
        version,
        COUNT(DISTINCT user) as player
    FROM 
        data
    WHERE 
        user IN (
            SELECT  
                user
            FROM 
                data
            WHERE 
                event_name = 'tutorial' AND quantity = -2) 
            AND 
                level = 2 AND win = 0
    GROUP BY 
        version
) as lost
ON tut_complete.version = lost.version
''')

,version,num_of_lv_2_player,lv_2_lost_player,percentage
0,1.5.2,6223,56,0.90
1,1.6.0,6396,48,0.75


In [5]:
df_152 = sqldf('''
    SELECT 
        day_diff,
        COUNT(DISTINCT user) num_of_player
    FROM 
        data 
    WHERE 
        version = '1.5.2'
    GROUP BY 
        day_diff
''')

df_160 = sqldf('''
    SELECT 
        day_diff,
        COUNT(DISTINCT user) num_of_player
    FROM 
        data 
    WHERE 
        version = '1.6.0'
    GROUP BY 
        day_diff
''')

In [6]:
player_count_152 = data[data['version'] == '1.5.2'].groupby('user')['day_diff'].max().value_counts().sum() 
player_count_160 = data[data['version'] == '1.6.0'].groupby('user')['day_diff'].max().value_counts().sum()

print(player_count_152, ' ', player_count_160)

6671   6929


In [7]:
retention_rate = pd.DataFrame({
    'ver_152': df_152['num_of_player'],
    'retention_rate_152' : round(df_152['num_of_player'] / player_count_152 * 100, 2),
    'ver_160': df_160['num_of_player'],
    'retention_rate_160' : round(df_160['num_of_player'] / player_count_160 * 100, 2)
})

retention_rate

,ver_152,retention_rate_152,ver_160,retention_rate_160
0,6663,99.88,6903,99.62
1,1780,26.68,1981,28.59
2,891,13.36,1036,14.95
3,517,7.75,714,10.30
4,385,5.77,520,7.50
5,275,4.12,423,6.10
6,202,3.03,334,4.82
7,183,2.74,289,4.17


In [8]:
max_lv_152 = data[data['version'] == '1.5.2'].groupby('user')['level'].max().describe()

In [9]:
max_lv_160 = data[data['version'] == '1.6.0'].groupby('user')['level'].max().describe()

In [10]:
max_lv = pd.DataFrame({
    'stat' : max_lv_152.index,
    'lv_152' : max_lv_152.values,
    'lv_160' : max_lv_160.values
})

max_lv

,stat,lv_152,lv_160
0,count,6671.000000,6929.000000
1,mean,9.414780,10.321691
2,std,17.068243,19.521062
3,min,1.000000,1.000000
4,25%,3.000000,4.000000
5,50%,6.000000,6.000000
6,75%,11.000000,11.000000
7,max,376.000000,519.000000


In [11]:
df_playtime = sqldf('''
SELECT 
    user,
    SUM(quantity) as play_time,
    version
FROM 
    data 
WHERE 
    event_name = 'game_end' 
GROUP BY 
    user 
''')

In [12]:
playtime_152 = df_playtime[df_playtime['version'] == '1.5.2'].describe()
playtime_160 = df_playtime[df_playtime['version'] == '1.6.0'].describe()

In [13]:
playtime = pd.DataFrame({
    'ver_152' : playtime_152['play_time'],
    'ver_160' : playtime_160['play_time']
})
playtime

,ver_152,ver_160
count,6435.000000,6607.000000
mean,779.674903,988.136673
std,3482.261223,4342.895857
min,6.000000,7.000000
25%,61.000000,69.000000
50%,133.000000,153.000000
75%,372.000000,451.000000
max,97182.000000,118078.000000


In [14]:
df_ue = sqldf('''
SELECT 
    user,
    COUNT(event_name) as user_engagement,
    version
FROM 
    data 
WHERE 
    event_name = 'user_engagement' 
GROUP BY 
    user 
''')

In [15]:
ue_152 = df_ue[df_ue['version'] == '1.5.2'].describe()
ue_160 = df_ue[df_ue['version'] == '1.6.0'].describe()

In [16]:
ue = pd.DataFrame({
    'ver_152' : ue_152['user_engagement'],
    'ver_160' : ue_160['user_engagement']
})
ue

,ver_152,ver_160
count,6671.000000,6911.000000
mean,13.515815,15.350166
std,35.046254,34.540717
min,1.000000,1.000000
25%,3.000000,4.000000
50%,6.000000,7.000000
75%,13.000000,14.000000
max,1276.000000,952.000000
